In [2]:
from gevent import monkey
monkey.patch_all()
import gevent
from gevent import pool
import json
import logging
import socket
import numpy as np
import simtk.unit as unit
from jinja2 import Template
import matplotlib.pyplot as plt
import evb

The history saving thread hit an unexpected error (This operation would block forever
	Hub: <Hub '' at 0x1d407931b70 default pending=0 ref=0 thread_ident=0x4238>
	Handles:
[HandleState(handle=<cdata 'struct uv_handle_s *' 0x000001D4085ABFF8>, type=b'check', watcher=<loop at 0x1d40788e588 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x000001D407F44F48>, type=b'timer', watcher=<loop at 0x1d40788e588 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x000001D4085AC208>, type=b'prepare', watcher=<loop at 0x1d40788e588 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x000001D4085AC838>, type=b'check', watcher=<loop at 0x1d40788e588 default pending=0 ref=0>, ref=1, active=0, closing=0)]).History will not be written to the database.


C:\Users\xinyan\Miniconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [3]:
class EVBClient(object):
    """
    The client of EVBHalmitonian.
    Have similar interfaces with evb.EVBHalmitonian.
    Support coroutine.
    """

    def __init__(self, port_list=[]):
        self.port_list = port_list
        self.pi = 0

    def initialize(self, conf):
        for pt in self.port_list:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect(("127.0.0.1", pt))
            data = "INIT" + json.dumps(conf)
            s.send(data.encode("utf-8"))
            ret = s.recv(1024).decode("utf-8")
            if ret == "FINISH":
                ans = 0
            else:
                ans = 1
                break
            s.close()
        return ans

    def calcEnergy(self, xyz):
        xyz_no_unit = xyz.value_in_unit(unit.angstrom)
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        pt = self.port_list[self.pi % len(self.port_list)]
        self.pi += 1
        s.connect(("127.0.0.1", pt))
        data = "ENER" + " ".join("%16.8f" % i for i in xyz_no_unit.ravel())
        s.send(data.encode("utf-8"))
        ret = s.recv(1024).decode("utf-8")
        if ret == "ERROR":
            ans = 1
        else:
            ans = 0
        s.close()
        return ans, unit.Quantity(float(ret), unit.kilojoule_per_mole)

    def calcEnergyGrad(self, xyz):
        xyz_no_unit = xyz.value_in_unit(unit.angstrom)
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        pt = self.port_list[self.pi % len(self.port_list)]
        self.pi += 1
        s.connect(("127.0.0.1", pt))
        data = "GRAD" + " ".join("%16.8f" % i for i in xyz_no_unit.ravel())
        s.send(data.encode("utf-8"))
        buff = []
        while True:
            d = s.recv(1024)
            buff.append(d)
            if len(d) < 1024:
                break
        ret = b"".join(buff).decode("utf-8")
        if ret == "ERROR":
            ans = 1
        else:
            ans = 0
        s.close()
        ret = np.array([float(i) for i in ret.strip().split()])
        return ans, unit.Quantity(ret[0], unit.kilojoule_per_mole), unit.Quantity(ret[1:].reshape((-1, 3)), unit.kilojoule_per_mole / unit.angstrom)


C:\Users\xinyan\Miniconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [15]:
TEMPFILE = "conf.temp"
var = np.array([ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
                 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 0.00000000e+00,  0.00000000e+00,  0.00000000e+00])
from fit import *
HESSFILE = "freq.fchk"
xyz, hess = getCHKHess(HESSFILE)
mass = getCHKMass(HESSFILE)

with open(TEMPFILE, "r") as f:
    template = Template("".join(f))

conf = json.loads(template.render(var=var))

C:\Users\xinyan\Miniconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [17]:
mass_mat = np.diag(1. / np.sqrt(mass.value_in_unit(unit.amu)))
hess_v = hess.value_in_unit(unit.kilocalorie_per_mole / unit.angstrom ** 2)
theta = np.dot(mass_mat, np.dot(hess_v, mass_mat))
qe, qv = np.linalg.eig(theta)
qvI = np.linalg.inv(qv)
theta_p = np.dot(qvI, np.dot(theta, qv))
dx = 0.00001

# gen halmitonian
client = EVBClient([5000,5001,5002,5003])
ret = client.initialize(conf)
# calc hess (unit in kJ / mol / A^2)
oxyz = xyz.value_in_unit(unit.angstrom).ravel()
dxyz = np.eye(oxyz.shape[0])
calc_hess = np.zeros(dxyz.shape)

def func(gi, hmat):
    txyz = unit.Quantity(value=(oxyz + dxyz[:, gi] * dx).reshape((-1, 3)), unit=unit.angstrom)
    _, tep, tgp = client.calcEnergyGrad(txyz)
    tgp = tgp.value_in_unit(unit.kilocalorie_per_mole / unit.angstrom).ravel()
# parallel
    txyz = unit.Quantity(value=(oxyz - dxyz[:, gi] * dx).reshape((-1, 3)), unit=unit.angstrom)
    _, ten, tgn = client.calcEnergyGrad(txyz)
    tgn= tgn.value_in_unit(unit.kilocalorie_per_mole / unit.angstrom).ravel()
    hmat[:, gi] = (tgp - tgn) / 2.0 / dx

#for gi in range(dxyz.shape[0]):
#    func(gi, calc_hess)

#for gi in range(dxyz.shape[0]):
#    pl.spawn(func(gi))
#pl.join()

gevent.joinall([gevent.spawn(func, gi, calc_hess) for gi in range(dxyz.shape[0])])

calc_theta = np.dot(mass_mat, np.dot(calc_hess, mass_mat))
# change basis
calc_theta_p = np.dot(qvI, np.dot(calc_theta, qv))

vib_qm, vib_mm = np.diag(theta_p), np.diag(calc_theta_p)
vib_qm = unit.Quantity(
    vib_qm, unit.kilocalorie_per_mole / unit.angstrom ** 2 / unit.amu)
vib_mm = unit.Quantity(
    vib_mm, unit.kilocalorie_per_mole / unit.angstrom ** 2 / unit.amu)
vib_qm = vib_qm.value_in_unit(
    unit.joule / unit.meter ** 2 / unit.kilogram)
vib_mm = vib_mm.value_in_unit(
    unit.joule / unit.meter ** 2 / unit.kilogram)
vib_qm = np.sqrt(np.abs(vib_qm)) / 2. / np.pi / \
    2.99792458e10 * np.sign(vib_qm)
vib_mm = np.sqrt(np.abs(vib_mm)) / 2. / np.pi / \
    2.99792458e10 * np.sign(vib_mm)

var = (calc_theta_p - theta_p) ** 2
var_diag = (((vib_qm - vib_mm) / vib_qm) ** 2).sum() / vib_mm.shape[0]
var_offdiag = (var - np.diag(np.diag(var))).sum() / \
    (var.shape[0] ** 2 - var.shape[0])
print(var_diag, var_offdiag)

4.689283778614833 4.7779019029638725


C:\Users\xinyan\Miniconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [35]:
(((vib_qm - vib_mm) / vib_qm) ** 2).sum()

1083.2245528600263

C:\Users\xinyan\Miniconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [38]:
((sorted(vib_qm) - vib_mm) / vib_qm) ** 2

array([1.04981104e+00, 9.94184451e-01, 9.48697596e-01, 9.79681210e-01,
       9.30773209e-01, 1.13216199e+00, 9.24137880e-01, 1.00851393e+00,
       9.12443104e-01, 9.03105480e-01, 9.91557687e-01, 9.04940584e-01,
       8.78062388e-01, 9.25216343e-01, 9.27371400e-01, 9.27766687e-01,
       9.26632468e-01, 9.45754497e-01, 9.38294813e-01, 9.68330844e-01,
       9.11380864e-01, 9.52666505e-01, 9.25609133e-01, 9.41924194e-01,
       9.02911447e-01, 9.20689894e-01, 9.12300744e-01, 9.41684856e-01,
       8.46470329e-01, 9.38582283e-01, 8.70348270e-01, 8.62369333e-01,
       8.26320660e-01, 8.59488133e-01, 8.58698884e-01, 8.57257180e-01,
       8.24417913e-01, 8.11402794e-01, 8.46278044e-01, 8.82385505e-01,
       8.24708643e-01, 7.34295375e-01, 4.20194422e-01, 3.48217518e-01,
       3.71688363e-01, 5.56012359e-01, 6.36341496e-01, 6.43997520e-01,
       5.82896668e-01, 3.69194780e-01, 4.15198011e-01, 4.41889189e-01,
       4.62027840e-01, 3.41044253e-01, 3.59501061e-01, 4.14453585e-01,
      

C:\Users\xinyan\Miniconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [44]:
vib_mm

array([ 3.67742059e+03,  3.28437539e+03,  3.20328509e+03,  3.24420288e+03,
        3.14831093e+03,  3.07618200e+03,  3.11108671e+03,  3.26178426e+03,
        3.10621826e+03,  3.07897667e+03,  3.23145501e+03,  3.09568347e+03,
        3.04170249e+03,  3.13030991e+03,  3.12847182e+03,  2.96657162e+03,
        3.01192348e+03,  3.16417841e+03,  3.14402276e+03,  3.21750442e+03,
        3.11871554e+03,  3.20169933e+03,  3.14821136e+03,  3.17788275e+03,
        3.13966860e+03,  3.14481812e+03,  3.15488557e+03,  3.21607035e+03,
        3.06812627e+03,  3.21922113e+03,  3.09002058e+03,  3.09513022e+03,
        3.05095811e+03,  3.11198231e+03,  3.12727158e+03,  3.11383999e+03,
        3.06827563e+03,  3.05405748e+03,  3.11332375e+03,  3.18387300e+03,
        3.11518809e+03,  1.81929353e+03,  1.43434429e+03,  1.33422578e+03,
        1.35666958e+03,  1.57131556e+03,  1.67030632e+03,  1.68365180e+03,
        1.62250520e+03,  1.36686057e+03,  1.42119162e+03,  1.45399629e+03,
        1.49255414e+03,  

C:\Users\xinyan\Miniconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
